In [1]:
from pippi.soundbuffer import SoundBuffer
from pippi import dsp,fx
import numpy as np
import pandas as pd
import sounddevice as sd
import helpers as hp
import param_generation as pg
import _pickle as pickle
from IPython.display import Audio
import librosa
import librosa.feature
sr=44100


/home/asalimi/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/asalimi/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method

In [2]:
class StackParams():
    def __init__(self,name="pset"):
        self.oscType=rd.choice([0,1,2],p=[0.8,0.1,0.1])
        self.isNoise=rd.choice([0,1],p=[0.5,0.5])     
        self.A=rd.randint(len(a_d_s_r))
        self.D=rd.randint(len(a_d_s_r))
        self.S=rd.randint(len(a_d_s_r))
        self.R=rd.randint(len(a_d_s_r)) 
        self.pitch_0=rd.choice(p0_pitches)
        self.pitch_1=rd.choice(p1_pitches)
        self.pitch_2=rd.choice(p2_pitches)
        self.pitch_3=rd.choice(p3_pitches) 
        self.bpCutLow=rd.choice(bp_pitches)
        self.bpCutHigh=rd.randint(self.bpCutLow,num_notes)
        self.bpOrder=rd.randint(len(filter_orders))
        self.amplitude=rd.randint(len(amplitudes))
        self.start=rd.randint(start_spacing)
        self.length=rd.randint(start_spacing-self.start)
    def getOscType(self):
        return osc_types[self.oscType]
    def getPitches(self):
        return all_pitches[self.pitch_0],all_pitches[self.pitch_1],all_pitches[self.pitch_2],all_pitches[self.pitch_3]
    def getBandPass(self):
        return all_pitches[self.bpCutLow],all_pitches[self.bpCutHigh],filter_orders[self.bpOrder]
    def getAmp(self):
        return amplitudes[self.amplitude]
    def getLength(self):
        return starts[self.length]+min_length
    def getStart(self):
        return lengths[self.start]

In [3]:
%%time
#generate 1 sound
import imp
imp.reload(pg)
imp.reload(hp)

out = dsp.buffer(length=1,channels=1)
params = []
stack_num = np.random.choice([2])
for i in range(stack_num): 
    p = pg.StackParams()
#     print(vars(p))
#     print(p.getStart())
    s = pg.Synth(p)
    out.dub(s.buff,p.getStart())
    params.append(p)
out = fx.norm(out,1)

# z=librosa.feature.rms(hp.memToAud(out))

# print(np.sum(z),hp.memToAud(out).shape)
# hp.specShow(out)
Audio(hp.memToAud(out),rate=sr, autoplay=True)


CPU times: user 22.6 ms, sys: 2.38 ms, total: 25 ms
Wall time: 24.3 ms


In [4]:
p.__dict__

{'oscType': 0,
 'isNoise': 1,
 'A': 2,
 'D': 1,
 'S': 0,
 'R': 1,
 'pitch_0': 80,
 'pitch_1': 108,
 'pitch_2': 0,
 'pitch_3': 16,
 'bpCutLow': 66,
 'bpCutHigh': 96,
 'bpOrder': 2,
 'amplitude': 2,
 'start': 1,
 'length': 4}